# Saliency map

In this task you have a trained classifier and you want to see what parts of the image are the most important ones for the classifier to predict selected class. One of the simplest method to do this is to occlude some parts of the image and see how big is the drop in the class probability compared to the probability for the original image.

We want you to create a heatmap of those occlusions (2D np.array called `heatmap`). Take the original image and replace squares of size 20x20 centered at point (i,j) with a value of your choice. Then pass the image trough the network and calculate the drop in probability class. Do this on a grid with stride 10 ((0,0), (0,10), ..., (10,0), (10,10), ..., (220,220)), so your heatmap is a matrix with 23 rows and 23 columns.

You have to write a code to calculate this heatmap, that runs under 2 minutes.


###### Definitions


In [2]:
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from PIL import Image
import requests
import numpy as np
import torch
from torchvision import transforms
import matplotlib.pyplot as plt


def get_image():
    url = "http://www.mimuw.edu.pl/~cygan/welonka.jpg"
    im = Image.open(requests.get(url, stream=True).raw)
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
    size = 224
    transformations = transforms.Compose(
        [
            transforms.Resize((size, size)),
            transforms.ToTensor(),
            normalize,
        ]
    )
    img_tensor = transformations(im.convert("RGB")).unsqueeze(0)
    return img_tensor


def get_model():
    model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
    model.eval()
    return model


def plot_image(img_tensor):
    img_tensor_c = img_tensor.clone()
    img_tensor_c -= img_tensor_c.min()
    img_tensor_c /= img_tensor_c.max()
    plt.imshow(img_tensor_c[0].permute(1, 2, 0))
    # plt.show()


def plot_heatmap(heatmap):
    plt.matshow(heatmap)
    plt.colorbar()
    plt.show()

#Your code


In [ ]:
def generate_heatmap(model, img_tensor):
    # YOUR CODE STARTS HERE
    # first calculate probability for original image
    output = model.forward(img_tensor)
    probs = torch.softmax(output, dim=1)

    goldfish_prob_baseline = probs[0, 1]
    heatmap = np.zeros(img_tensor.shape[2:])

    STEP_SIZE = 16

    for i in range(0, img_tensor.shape[2], STEP_SIZE):
        for j in range(0, img_tensor.shape[3], STEP_SIZE):
            masked_imgs = torch.repeat_interleave(img_tensor, STEP_SIZE**2, 0)

            for i_in in range(STEP_SIZE):
                for j_in in range(STEP_SIZE):
                    masked_imgs[i_in * STEP_SIZE + j_in, :, i + i_in, j + j_in] = 0

            output = model.forward(masked_imgs)
            probs = torch.softmax(output, dim=1)

            for i_in in range(STEP_SIZE):
                for j_in in range(STEP_SIZE):
                    heatmap[i + i_in, j + j_in] = (
                        goldfish_prob_baseline - probs[i_in * STEP_SIZE + j_in, 1]
                    )

    # YOUR CODE ENDS HERE
    return heatmap

#Test


In [ ]:
img_tensor = get_image()
model = get_model()
heatmap = generate_heatmap(model, img_tensor)

plot_image(img_tensor)
plot_heatmap(heatmap)